# Data Cleaning

### Missing value imputation using Sciket-Learn

### Different strategy for different variables (Numerical & Categorical) with Scikit-Learn

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
# Load data
train = pd.read_csv('https://raw.githubusercontent.com/Bhushan0130/Datasets/main/houseprice_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Bhushan0130/Datasets/main/houseprice_test.csv')

In [3]:
print('Shape of train: ', train.shape)
print('Shape of train: ', test.shape)

Shape of train:  (1460, 81)
Shape of train:  (1459, 80)


In [4]:
x_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]
print('Shape of x_train: ', x_train.shape)
print('Shape of y_train: ', y_train.shape)

Shape of x_train:  (1460, 80)
Shape of y_train:  (1460,)


## Missing Value Imputation 

In [5]:
isnull_var = x_train.isnull().sum()
isnull_var

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64

In [6]:
num_var = x_train.select_dtypes (include = ['float', 'int']).columns
num_var_null = [var  for var in num_var if isnull_var[var] > 0]
num_var_null

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [7]:
cat_var = x_train.select_dtypes(include = 'object')
cat_var_null = [var for var in cat_var if isnull_var[var] > 0]
cat_var_null

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [8]:
num_apply_mean = ['LotFrontage']
num_apply_median = ['MasVnrArea', 'MasVnrArea']
cat_apply_mode = ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
                  'BsmtFinType2', 'Electrical', 'FireplaceQu']
cat_apply_missing = ['GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']


In [9]:
mean_pipeline = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'mean'))])
median_pipeline = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'median'))])
mode_pipeline = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'most_frequent'))])
constant_pipeline = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'constant', fill_value = 'M'))])

In [10]:
preprocessor = ColumnTransformer(transformers = [('mean_tran' , mean_pipeline, num_apply_mean),
                                 ('median_tran', median_pipeline, num_apply_median),
                                 ('mode_tran', mode_pipeline, cat_apply_mode),
                                  ('constant_tran',constant_pipeline, cat_apply_missing) ])

In [11]:
preprocessor.fit(x_train)

ColumnTransformer(transformers=[('mean_tran',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_tran',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'MasVnrArea']),
                                ('mode_tran',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('constant_tran',
                                 Pipeline(steps=[

In [15]:
preprocessor.transform # to know about what value imputed

<bound method ColumnTransformer.transform of ColumnTransformer(transformers=[('mean_tran',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_tran',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'MasVnrArea']),
                                ('mode_tran',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('constant_tran',
    

In [16]:
preprocessor.named_transformers_['mean_tran'].named_steps['imputer'].statistics_

array([70.04995837])

In [17]:
train['LotFrontage'].mean()

70.04995836802665

In [18]:
preprocessor.named_transformers_['mode_tran'].named_steps['imputer'].statistics_

array(['Grvl', 'None', 'TA', 'TA', 'No', 'Unf', 'Unf', 'SBrkr', 'Gd'],
      dtype=object)

In [20]:
x_train_clean = preprocessor.transform(x_train)
test_clean = preprocessor.transform(test)

In [21]:
x_train_clean

array([[65.0, 196.0, 196.0, ..., 'M', 'M', 'M'],
       [80.0, 0.0, 0.0, ..., 'M', 'M', 'M'],
       [68.0, 162.0, 162.0, ..., 'M', 'M', 'M'],
       ...,
       [66.0, 0.0, 0.0, ..., 'M', 'GdPrv', 'Shed'],
       [68.0, 0.0, 0.0, ..., 'M', 'M', 'M'],
       [75.0, 0.0, 0.0, ..., 'M', 'M', 'M']], dtype=object)

In [24]:
preprocessor.transformers_ # drop value is present

[('mean_tran',
  Pipeline(steps=[('imputer', SimpleImputer())]),
  ['LotFrontage']),
 ('median_tran',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))]),
  ['MasVnrArea', 'MasVnrArea']),
 ('mode_tran',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))]),
  ['Alley',
   'MasVnrType',
   'BsmtQual',
   'BsmtCond',
   'BsmtExposure',
   'BsmtFinType1',
   'BsmtFinType2',
   'Electrical',
   'FireplaceQu']),
 ('constant_tran',
  Pipeline(steps=[('imputer',
                   SimpleImputer(fill_value='M', strategy='constant'))]),
  ['GarageType',
   'GarageFinish',
   'GarageQual',
   'GarageCond',
   'PoolQC',
   'Fence',
   'MiscFeature']),
 ('remainder',
  'drop',
  [0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,


In [27]:
clean_x_train_null= pd.DataFrame(x_train_clean, columns = num_apply_mean+ num_apply_median+ cat_apply_mode+ cat_apply_missing)

In [28]:
clean_x_train_null.head(2)

,LotFrontage,MasVnrArea,MasVnrArea,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65.0,196.0,196.0,Grvl,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,M,M,M
1,80.0,0.0,0.0,Grvl,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,M,M,M


In [29]:
clean_x_train_null.isnull().sum().sum()

0

In [40]:
clean_x_train_null.shape

(1460, 19)

In [42]:
clean_x_train.columns

Index(['LotFrontage', 'MasVnrArea', 'MasVnrArea', 'Alley', 'MasVnrType',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'],
      dtype='object')

In [32]:
# testing to accurate value imputation 
# test['Alley'].mode()
test ['Alley'].value_counts()

Grvl    70
Pave    37
Name: Alley, dtype: int64

In [34]:
x_train.isnull().sum().sum()

6965

In [38]:
isnull_var

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64

In [43]:
col = ['LotFrontage', 'MasVnrArea', 'MasVnrArea', 'Alley', 'MasVnrType',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']

In [46]:
clean_x_train_null.head()

,LotFrontage,MasVnrArea,MasVnrArea,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65.0,196.0,196.0,Grvl,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,M,M,M
1,80.0,0.0,0.0,Grvl,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,M,M,M
2,68.0,162.0,162.0,Grvl,BrkFace,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,M,M,M
3,60.0,0.0,0.0,Grvl,None,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,Unf,TA,TA,M,M,M
4,84.0,350.0,350.0,Grvl,BrkFace,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,M,M,M


In [53]:
# Create Clean x_train, DataFrame with call variables

In [55]:
 remainder_var_index=[0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   59,
   61,
   62,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   75,
   76,
   77,
   78,
   79]

In [58]:
remainder_var = [isnull_var.keys()[var_index] for var_index in remainder_var_index]
remainder_var

['Id',
 'MSSubClass',
 'MSZoning',
 'LotArea',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SaleType',
 'SaleCondition']

In [59]:
len(remainder_var_index)

62

In [60]:
# x_train = pd.concat([clean_x_train_null, train[remainder_var_index] ], axis = 1)

KeyError: "None of [Int64Index([ 0,  1,  2,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,\n            19, 20, 21, 22, 23, 24, 27, 28, 29, 34, 36, 37, 38, 39, 40, 41, 43,\n            44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 59, 61, 62, 65,\n            66, 67, 68, 69, 70, 71, 75, 76, 77, 78, 79],\n           dtype='int64')] are in the [columns]"